In [1]:
% pip install opencv-contrib-python

In [2]:
% pip install opencv-python --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from PIL import Image

In [2]:
def display(np_image):
    """
    This is a display function that we have added to show numpy images at full size
    If you pass in an image with 3 channels, it will be displayed in RGB
    If you passn in an image with 1 channel, it will be displayed in grayscale
    """
    dpi = matplotlib.rcParams['figure.dpi']
    if len(np_image.shape) == 3:
        height, width, depth = np_image.shape
    else:
        height, width = np_image.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image in either RGB or grayscale (depending on the amount of dimensions)
    if (len(np_image.shape) >= 3):
        ax.imshow(np_image)
    else:
        ax.imshow(np_image, cmap='gray')

    plt.show()

Test on image

In [ ]:
net = cv2.dnn_DetectionModel('yolov4-tiny-custom29.cfg', 'yolov4-tiny-custom29_final.weights')
net.setInputSize(416, 416)
net.setInputScale(1.0 / 255)
net.setInputSwapRB(True)

frame = cv2.imread('./test-pig.jpg')

with open('obj.names', 'rt') as f:
    names = f.read().rstrip('\n').split('\n')

classes, confidences, boxes = net.detect(frame, confThreshold=0.1, nmsThreshold=0.4)

for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
    left, top, width, height = box
    plt.imshow(frame[top:top + height, left:left + width])

for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
    label = '%.2f' % confidence
    label = '%s: %s' % (names[classId], label)
    labelsize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    left, top, width, height = box
    top = max(top, labelsize[1])
    cv2.rectangle(frame, box, color=(2, 255, 0), thickness=2)
    cv2.rectangle(frame, (left, top - labelsize[1]), (left + labelsize[0], top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

cv2.imshow('out', frame)

# destroy window
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


Test on video

In [4]:
net = cv2.dnn_DetectionModel('yolov4-tiny-custom29.cfg', 'Copy_of_yolov4-tiny-custom29_best.weights')
net.setInputSize(416, 416)
net.setInputScale(1.0 / 255)
net.setInputSwapRB(True)

tail_ear = cv2.dnn_DetectionModel('taiL_ear.cfg', 'taiL_ear.weights')
tail_ear.setInputSize(416, 416)
tail_ear.setInputScale(1.0 / 255)
tail_ear.setInputSwapRB(True)

tail_classifier = keras.models.load_model('tail-classifier.hdf5')

In [1]:
def predict_tail(model_, img):
    pred_classes = ['curly', 'straight']
    input_im = cv2.resize(img, (224, 224), interpolation=cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1, 224, 224, 3)
    predictions = model_.predict(input_im)

    closest_guess = None
    index_guess = 0
    prob_guess = 0

    for pre in predictions[0]:
        if pre > prob_guess:
            prob_guess = pre
            closest_guess = index_guess
        index_guess = index_guess + 1

    return pred_classes[closest_guess]

In [17]:
video = 'Cam2-3.mp4'

cap = cv2.VideoCapture(video)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_before.avi', fourcc, 20.0, size)

frame_number = 0

with open('obj.names', 'rt') as f:
    names = f.read().rstrip('\n').split('\n')

while cap.isOpened():
    ret, frame = cap.read()
    frame_number += 1
    counter = 0
    tail_dict = {}

    if ret:
        classes, confidences, boxes = net.detect(frame, confThreshold=0.45, nmsThreshold=0.45)
        pig_index = 0
        display_index = 0
        for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
            counter = counter + 1
            left, top, width, height = box
            image = frame[top:top + height, left:left + width]
            classes_te, confidences_te, boxes_te = tail_ear.detect(image, confThreshold=0.4, nmsThreshold=0.4)

            for classId_tail_ear, box_tail in zip(classes_te, boxes_te):
                if classId_tail_ear == 0:  #tail
                    left, top, width, height = box_tail
                    tail = predict_tail(tail_classifier, image[top:top + height, left:left + width])
                    tail_dict[pig_index] = tail

            pig_index = pig_index + 1

        for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
            label = '%.2f' % confidence
            if display_index in tail_dict.keys():
                label = '%s(%s tail): %s' % (names[classId], tail_dict[display_index], label)
            else:
                label = '%s(tail not visible): %s' % (names[classId], label)
            labelsize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            left, top, width, height = box
            top = max(top, labelsize[1])
            cv2.rectangle(frame, box, color=(2, 255, 0), thickness=2)
            cv2.rectangle(frame, (left, top - labelsize[1]), (left + labelsize[0], top + baseLine), (255, 255, 255),
                          cv2.FILLED)
            cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
            display_index = display_index + 1

        out.write(frame)
        # cv2.namedWindow('Objet Detection YOLO',cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('Objet Detection YOLO', 1600,900)
        # cv2.imshow("Objet Detection YOLO", frame)

        if cv2.waitKey(1) == 13:  #13 is the Enter Key
            break
    else:
        print(frame_number)
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

34


-1

combine 2 classifiers

In [5]:
net = cv2.dnn_DetectionModel('yolov4-tiny-custom29.cfg', 'Pig_detection_model.weights')
net.setInputSize(416, 416)
net.setInputScale(1.0 / 255)
net.setInputSwapRB(True)

tail_ear = cv2.dnn_DetectionModel('tail+ear.cfg', 'tails+ear.weights')
tail_ear.setInputSize(416, 416)
tail_ear.setInputScale(1.0 / 255)
tail_ear.setInputSwapRB(True)


<dnn_Model 00000225528EF210>

Logging function

In [2]:
#importing the module
import json
from datetime import datetime

Execute the model

In [3]:
class PigData:
    def __init__(self, pig_pos, pig_size, tail_type, tail_pos, tail_size, ear_pos, ear_size, ear_visible):
        self.pig_pos = pig_pos
        self.pig_size = pig_size
        self.tail_type = tail_type
        self.tail_size = tail_size
        self.tail_pos = tail_pos
        self.ear_pos = ear_pos
        self.ear_size = ear_size
        self.ear_visible = ear_visible

    def toJson(self):
        return json.dumps({'pig_pos': self.pig_pos, 'pig_size': self.pig_size,
                           'tail_type':self.tail_type, 'tail_size':self.tail_size,
                           'tail_pos':self.tail_pos, 'ear_pos': self.ear_pos, 'ear_size': self.ear_size, 'ear_visible': self.ear_visible
        }, sort_keys=True, indent=4)

    def __repr__(self):
        return self.toJson()

class Frame:
    def __init__(self, f_number, total_pig, pigs_data):
        self.f_number = f_number
        self.total_pig = total_pig
        self.pigs_data = pigs_data

    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

    def __repr__(self):
        return self.toJson()

class LogData:
    def __init__(self, width, height, time_stamp, frame):
        self.width = width
        self.height = height
        self.time_stamp = time_stamp
        self.frame = frame

    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

    def __repr__(self):
        return self.toJson()


In [6]:
video = 'pig_yolo_test.mov'
save_image_path = "crop-pig"

cap = cv2.VideoCapture(video)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, size)

frame_number = 0

with open('obj.names', 'rt') as f:
    names = f.read().rstrip('\n').split('\n')

logData = LogData(int(width), int(height), str(datetime.now()), [])

while cap.isOpened():
    ret, frame = cap.read()
    frame_number += 1
    counter = 0
    pig_dict = {}

    if ret:
        classes, confidences, boxes = net.detect(frame, confThreshold=0.45, nmsThreshold=0.45)
        pig_index = 0
        display_index = 0

        frame_log = Frame(frame_number,len(boxes), [])

        for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
            counter = counter + 1
            left, top, width, height = box
            image = frame[top:top + height, left:left + width]
            classes_te, confidences_te, boxes_te = tail_ear.detect(image, confThreshold=0.4, nmsThreshold=0.4)

            tail_na = 'not visible'
            pigData = PigData([int(left), int(top)], [int(width), int(height)], tail_na, [], [], [], [], False)

            for classId_tail_ear, box_tail_ear in zip(classes_te, boxes_te):
                if classId_tail_ear == 0:  #curly
                    pigData.tail_type = 'curly'
                    left, top, width, height = box_tail_ear
                    pigData.tail_pos = [int(pigData.pig_pos[0]+left), int(pigData.pig_pos[0]+top)]
                    pigData.tail_size = [int(width), int(height)]
                elif classId_tail_ear == 1:  #straight
                    pigData.tail_type = 'straight'
                    left, top, width, height = box_tail_ear
                    pigData.tail_pos = [int(pigData.pig_pos[0]+left), int(pigData.pig_pos[0]+top)]
                    pigData.tail_size = [int(width), int(height)]
                elif classId_tail_ear == 2:  #ear
                    pigData.ear_visible = True
                    left, top, width, height = box_tail_ear
                    pigData.ear_pos.append([int(pigData.pig_pos[0]+left), int(pigData.pig_pos[0]+top)])
                    pigData.ear_size.append([int(width), int(height)])

            for cl_te, con_te, box_te in zip(classes_te, confidences_te, boxes_te):
                label = '%.2f' % confidence
                box_color = None
                if cl_te == 0:  #curly
                    label = '(curly): %s' % label
                    box_color = (255, 0, 0)
                elif cl_te == 1:  #straight
                    label = '(straight): %s' % label
                    box_color = (255, 0, 0)
                elif cl_te == 2:  #ear
                    label = '(ear): %s' % label
                    box_color = (0, 0, 255)
                labelsize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                l, t, width, height = box_te
                left = pigData.pig_pos[0]+l
                top = pigData.pig_pos[1]+t
                top = max(top, labelsize[1])
                cv2.rectangle(frame, [left, top, width, height], color=box_color, thickness=2)
                cv2.rectangle(frame, (left, top - labelsize[1]), (left + labelsize[0], top + baseLine), (255, 255, 255), cv2.FILLED)
                cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (219, 0, 0))

            frame_log.pigs_data.append(pigData)
            logData.frame.append(frame_log)

            pig_dict[pig_index] = pigData
            pig_index = pig_index + 1

        for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
            label = '%.2f' % confidence
            if display_index in pig_dict.keys():
                label = '%s(%s tail): %s' % (names[classId], pig_dict[display_index].tail_type, label)
            else:
                label = '%s(tail not visible): %s' % (names[classId], label)
            labelsize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            left, top, width, height = box
            top = max(top, labelsize[1])
            cv2.rectangle(frame, box, color=(2, 255, 0), thickness=2)
            cv2.rectangle(frame, (left, top - labelsize[1]), (left + labelsize[0], top + baseLine), (255, 255, 255),
                          cv2.FILLED)
            cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
            display_index = display_index + 1

        jsonLog = json.loads(logData.toJson())

        with open(f"{video}.json", "w") as outfile:
            json.dump(jsonLog, outfile)

        out.write(frame)
        # cv2.namedWindow('Objet Detection YOLO',cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('Objet Detection YOLO', 1600,900)
        # cv2.imshow("Objet Detection YOLO", frame)

        if cv2.waitKey(1) == 13:  #13 is the Enter Key
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
#get back the webcam
cap.release()
cv2.destroyAllWindows()